In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import math
import ROOT
import numpy as np, pandas as pd
import yaml
import base64
import ctypes
import logging
import bokeh
import sys
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.Tools.compressArray import arrayCompressionRelative8

output_notebook()

In [ ]:
inputFile="/home/berki/Downloads/Merged.root"

In [ ]:
f=ROOT.TFile.Open(inputFile)
sampled = f.Get("sampled;1")
sampled.GetListOfBranches().ls()

In [ ]:
sampled.SetAlias("qPt","tracks.getQ2Pt()")
sampled.SetAlias("phi","tracks.getPhi()")
sampled.SetAlias("eta","tracks.getEta()")
sampled.SetAlias("tgl","tracks.getTgl()")
sampled.SetAlias("charge","tracks.getCharge()")
sampled.SetAlias("pzPt","tracks.mP[3]")

In [ ]:
df=tree2Panda(sampled, [".*"], "", exclude=[])

In [ ]:
df = df.rename(columns={column : column.replace("tracksExtra_tracksExtra_","").replace("collisions_collisions_","").replace("tracks_tracks_","") for column in df.columns})

In [ ]:
df.columns

In [ ]:
variables = list(df.columns)
[variables.remove(var) for var in variables if "[" in var]
variables.remove("fTOFExpMom") # inf in the list
variables.sort()
print(variables)

In [ ]:
parameterArray = [
    {"name": "size", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
    {"name": "legendVisible", "value":False},
    {"name": "nPointRender", "range":[0, 5000], "value": 1000},
    {"name": "transformZ", "value": "bin_count", "options": ["bin_count","log_bin_count"]},
    {"name": "varX", "value": variables[0], "options": variables},
    {"name": "varY", "value": variables[0], "options": variables},
    {"name": "nBinsX", "range":[0, 200], "value": 50},
    {"name": "nBinsY", "range":[0, 200], "value": 50},
    {"name": "rangeX", "value": [0, 1], "range": [0, 1]},
    {"name": "rangeY", "value": [0, 1], "range": [0, 1]},
]
widgetParams=[
      ['select', ['varX'], {"name":"varX"}],
      ['select', ['varY'], {"name":"varY"}],
      ['select', ['transformZ'], {"name":"transformZ"}],
      ['spinner',['nBinsX'], {"name":"nBinsX"}],
      ['spinner',['nBinsY'], {"name":"nBinsY"}],
      ['range',['rangeX'], {"name":"rangeX"}],
      ['range',['rangeY'], {"name":"rangeY"}],
      ['toggle',['legendVisible'], {"name":"legendVisible"}],
      ['slider',['size'], {"name":"size"}],
      ['textQuery', {"title": "Selection", "name":"selectionText"}], 
]

#widgetParamsSelect = [["range",[var], {"name":var}] for var in variables]
#widgetParams.extend(widgetParamsSelect)

widgetLayoutDesc={
    "Histogram:": [["varX","varY","transformZ"],["nBinsX","nBinsY"],['rangeX','rangeY']],
    "Visual:": [["legendVisible","size"]],
    "Selection:":[["selectionText"]]
    #"Selection:": [variables[x:x+5] for x in range(0, len(variables), 5)],
}

In [ ]:
histoArray = [
    {"name": "Hist1D", "variables": ["varX"],"nbins":"nBinsX"},
    {"name": "Hist2D", "variables": ["varX","varY"], "nbins":["nBinsX","nBinsY"]}
]

tooltips = []
aliasArray= [
    { "name": "log_bin_count","variables": ["bin_count"],"func": "return Math.log(Math.max(1,bin_count))","context": "Hist2D"}
    #{ "name": "log_bin_count","variables": ["bin_count"],"func": "return Math.log(bin_count)","context": "Hist2D"}
]

figureArray = [
    [['bin_center'], ['bin_count'], {"source":"Hist1D"}],
    [[('bin_bottom_0','bin_top_0')], [('bin_bottom_1','bin_top_1')], {"source":"Hist2D", "colorZvar":"transformZ"}],
    {"legend_options": {"visible":"legendVisible"}}
]

figureLayout = {
    "Histograms": [[0,1], {'plot_height':400,'sizing_mode':'scale_width'}],
}

output_file("testDashboard.html")
xxx = bokehDrawSA.fromArray(df, "indices>0", figureArray, widgetParams, layout=figureLayout, widgetLayout=widgetLayoutDesc,
                            histogramArray=histoArray,sizing_mode='scale_width',parameterArray=parameterArray,aliasArray=aliasArray,
                            arrayCompression=arrayCompressionRelative8)